# Project I - Finding the Higgs Boson

In [27]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from proj1_helpers import *
from learning_methods import *
from helper_functions import *
import matplotlib.pyplot as plt
import pandas
from pandas.tools.plotting import scatter_matrix

## Loading and pre-processing the data

In [13]:
# Loading of datasets

DATA_TRAIN = 'data/train.csv'
y, tx, ids = load_csv_data(DATA_TRAIN)

DATA_TEST = 'data/test.csv'
_, tX_test, ids_test = load_csv_data(DATA_TEST)

In [24]:
def ridge_regression_demo(x, y, degree, ratio, seed):
    """ridge regression demo."""
    # define parameter
    lambdas = np.logspace(-5, 0, 15)

    trainX, testX, trainY, testY = split_data(tx, y, ratio, seed=1)

    train_Matrix = build_poly(trainX, degree)
    test_Matrix = build_poly(testX, degree)

    rmse_tr = []
    rmse_te = []
    for ind, lambda_ in enumerate(lambdas):

        w, rmse = ridge_regression(trainY, train_Matrix, lambda_)
        rmse_tr = rmse_tr.append(rmse)
        rmse_te = rmse_te.append(np.sqrt((testY - w*test_Matrix)**2))
        print("proportion={p}, degree={d}, lambda={l:.3f}, Training RMSE={tr:.3f}, Testing RMSE={te:.3f}".format(
               p=ratio, d=degree, l=lambda_, tr=rmse_tr[ind], te=rmse_te[ind]))
        
    # Plot the obtained results
    #plot_train_test(rmse_tr, rmse_te, lambdas, degree)

In [31]:
# Gestion of the features
tx, _, _ = standardize(tx)
#tx = feature_handling(tx)

# For the moment, we separate our dataset between a training and a testing set. To Delete later
ratio = 0.7

seed = 56
degree = 7
split_ratio = 0.7
ridge_regression_demo(tx, y, degree, split_ratio, seed)

ValueError: setting an array element with a sequence.

## Machine Learning Magic

(175000, 30)

## Results preparation

In [ ]:
# Visualizations

In [ ]:
# Prepare final csv file with results